In [1]:
using GynC, Plots, Distributions
gr(grid=false)

Plots.GRBackend()

In [2]:
n = 200
mass = 70
xs = linspace(1,50,n) |> collect
tmeas = [1]
phi(k) = GynC.Federn.odesol(k, tmeas, m=mass)[1]
plot(xs, phi.(xs))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -9 
 
 
 -8 
 
 
 -7 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 21.7148,384.952 24.601,382.922 27.4871,380.893 30.3733,378.866 33.2595,376.84 36.1456,374.814 39.0318,372.79 41.918,370.768 44.8042,368.746 47.6903,366.726 
 50.5765,364.707 53.4627,362.689 56.3488,360.672 59.235,358.656 62.1212,356.642 65.0074,354.629 67.8935,352.617 70.7797,350.606 73.6659,348.596 76.5521,346.588 
 79.4382,344.58 82.3244,342.574 85.2106,340.569 88.0967,338.565 90.9829,336.563 93.8691,334.561 96.7553,332.561 99.6414,330.562 102.528,328.564 105.414,326.568 
 108.3,324.572 111.186,322.578 114.072,320.585 116.958,318.593 119.845,316.602 122.731,314.612 125.617,312.624 128.503,310.637 131.389,308.65 134.275,306.666 
 137.162,304.682 140.048,302.699 142.934,300.718 145.82,298.738 148.706,296.759 151.593,294.781 154.479,292.804 157.365,290.829 160.251,288.854 163.137,286.881 
 166.023,284.909 168.91,282.938 171.796,280.969 174.682,279 177.568,277.033 180.454,275.067 183.34,273.102 186.227,271.138 189.113,269.175 191.999,267.214 
 194.885,265.253 197.771,263.294 200.657,261.336 203.544,259.379 206.43,257.424 209.316,255.469 212.202,253.516 215.088,251.564 217.974,249.613 220.861,247.663 
 223.747,245.714 226.633,243.767 229.519,241.82 232.405,239.875 235.292,237.931 238.178,235.988 241.064,234.047 243.95,232.106 246.836,230.167 249.722,228.229 
 252.609,226.291 255.495,224.356 258.381,222.421 261.267,220.487 264.153,218.555 267.039,216.624 269.926,214.693 272.812,212.764 275.698,210.837 278.584,208.91 
 281.47,206.985 284.356,205.06 287.243,203.137 290.129,201.215 293.015,199.294 295.901,197.374 298.787,195.456 301.673,193.538 304.56,191.622 307.446,189.707 
 310.332,187.793 313.218,185.88 316.104,183.968 318.99,182.058 321.877,180.149 324.763,178.24 327.649,176.333 330.535,174.427 333.421,172.523 336.308,170.619 
 339.194,168.716 342.08,166.815 344.966,164.915 347.852,163.016 350.738,161.118 353.625,159.221 356.511,157.326 359.397,155.431 362.283,153.538 365.169,151.646 
 368.055,149.754 370.942,147.865 373.828,145.976 376.714,144.088 379.6,142.202 382.486,140.316 385.372,138.432 388.259,136.549 391.145,134.667 394.031,132.786 
 396.917,130.907 399.803,129.028 402.689,127.151 405.576,125.275 408.462,123.399 411.348,121.526 414.234,119.653 417.12,117.781 420.007,115.91 422.893,114.041 
 425.779,112.173 428.665,110.306 431.551,108.44 434.437,106.575 437.324,104.711 440.21,102.848 443.096,100.987 445.982,99.1264 448.868,97.2672 451.754,95.4091 
 454.641,93.5521 457.527,91.6963 460.413,89.8416 463.299,87.988 466.185,86.1356 469.071,84.2843 471.958,82.4341 474.844,80.585 477.73,78.7371 480.616,76.8903 
 483.502,75.0446 486.388,73.2 489.275,71.3566 492.161,69.5143 495.047,67.6731 497.933,65.833 500.819,63.9941 503.705,62.1563 506.592,60.3196 509.478,58.484 
 512.364,56.6496 515.25,54.8163 518.136,52.984 521.023,51.153 523.909,49.323 526.795,47.4942 529.681,45.6664 532.567,43.8398 535.453,42.0144 538.34,40.19 
 541.226,38.3668 544.112,36.5446 546.998,34.7236 549.884,32.9037 552.77,31.0849 555.657,29.2673 558.543,27.4507 561.429,25.6353 564.315,23.821 567.201,22.0078 
 570.087,20.1957 572.974,18.3847 575.86,16.5749 578.746,14.7661 581.632,12.9585 584.518,11.152 587.404,9.34657 590.291,7.54227 593.177,5.73908 596.063,3.93701 
 
 "/>
 
 
 
 
 y1

In [121]:
n = 200
mass = 70
tmeas = [1]
measerr = Normal(0,.3)
ndata = 50
augz = 100
reg = .95
augd = 10

xs = linspace(1,50,n) |> collect

phi(k) = GynC.Federn.odesol(k, tmeas, m=mass)[1]
plot(xs, phi.(xs))

w0 = ones(length(xs)) / length(xs);
prior = GynC.Federn.prior
wprior = pdf(prior, xs)
wprior = wprior / sum(wprior);

srand(3)
m = GynC.syntheticmodel(xs, phi, prior, ndata, augz, measerr)
ms = GynC.smoothedmodel(m, augd);

In [122]:
h = 0.1
niter = 1000

wsgrad = Dict()
@time begin
  wsgrad["NPMLE"] = GynC.em(m, w0, niter)
  wsgrad["DS-MLE"] = GynC.em(ms, w0, niter);
  wsgrad["MPLE"]  = GynC.mple(m, w0, niter, reg, h)
  wsgrad["Reference Prior"] = GynC.mple(m, w0, niter, 1, h);
end

ws = wsgrad

GynC.mple_obj(m, 0)(ws["NPMLE"][end]) |> println
GynC.mple_obj(ms, 0)(ws["DS-MLE"][end]) |> println
GynC.mple_obj(m, reg)(ws["MPLE"][end]) |> println
GynC.mple_obj(m, 1)(ws["Reference Prior"][end]) |> println

labels = ["NPMLE" "DS-MLE" "MPLE" "Reference Prior"]
densities = map(l->ws[l][end], labels) |> vec

plot(xs, wprior, label="True Prior", legendfont=font(10), tickfont=font(8), ylims=(0,0.05), size=(600, 350), grid=false)
plot!(xs, densities, labels=labels)

computing likelihood matrix (200x50) (0.001362378 seconds)


computing likelihood matrix (200x500) (0.00550396 seconds)


computing likelihood matrix (20000x200) (0.259696542 seconds)
computing likelihood matrix (50x200) (0.001176568 seconds)


 81.641537 seconds (98.31 k allocations: 1.377 GB, 0.82% gc time)
-37.823675679870306
computing likelihood matrix (500x200) (0.01048273 seconds)
-459.6604187475762
-1.0627964719948528
1.4771237790350695


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 0.05 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.7; stroke-opacity:1; fill:none" points="
 39.4926,334.952 42.2894,334.952 45.0862,334.952 47.8831,334.952 50.6799,334.952 53.4767,334.952 56.2736,334.952 59.0704,334.952 61.8673,334.952 64.6641,334.952 
 67.4609,334.952 70.2578,334.951 73.0546,334.951 75.8514,334.95 78.6483,334.949 81.4451,334.948 84.2419,334.945 87.0388,334.941 89.8356,334.935 92.6325,334.925 
 95.4293,334.909 98.2261,334.886 101.023,334.852 103.82,334.801 106.617,334.727 109.413,334.62 112.21,334.469 115.007,334.257 117.804,333.965 120.601,333.565 
 123.398,333.028 126.194,332.314 128.991,331.377 131.788,330.167 134.585,328.622 137.382,326.679 140.179,324.268 142.976,321.318 145.772,317.761 148.569,313.534 
 151.366,308.585 154.163,302.879 156.96,296.402 159.757,289.169 162.553,281.225 165.35,272.654 168.147,263.574 170.944,254.145 173.741,244.558 176.538,235.036 
 179.334,225.824 182.131,217.181 184.928,209.363 187.725,202.618 190.522,197.166 193.319,193.191 196.115,190.831 198.912,190.166 201.709,191.217 204.506,193.944 
 207.303,198.247 210.1,203.97 212.896,210.917 215.693,218.854 218.49,227.528 221.287,236.676 224.084,246.041 226.881,255.379 229.677,264.472 232.474,273.133 
 235.271,281.212 238.068,288.595 240.865,295.206 243.662,301.005 246.458,305.981 249.255,310.15 252.052,313.548 254.849,316.227 257.646,318.246 260.443,319.672 
 263.239,320.57 266.036,321.004 268.833,321.036 271.63,320.719 274.427,320.1 277.224,319.222 280.02,318.12 282.817,316.822 285.614,315.354 288.411,313.736 
 291.208,311.984 294.005,310.113 296.802,308.137 299.598,306.066 302.395,303.914 305.192,301.69 307.989,299.406 310.786,297.075 313.583,294.708 316.379,292.32 
 319.176,289.924 321.973,287.535 324.77,285.167 327.567,282.838 330.364,280.562 333.16,278.357 335.957,276.238 338.754,274.222 341.551,272.325 344.348,270.561 
 347.145,268.946 349.941,267.493 352.738,266.213 355.535,265.119 358.332,264.219 361.129,263.522 363.926,263.034 366.722,262.758 369.519,262.699 372.316,262.855 
 375.113,263.227 377.91,263.809 380.707,264.598 383.503,265.586 386.3,266.765 389.097,268.124 391.894,269.652 394.691,271.336 397.488,273.161 400.284,275.114 
 403.081,277.178 405.878,279.338 408.675,281.577 411.472,283.879 414.269,286.227 417.065,288.607 419.862,291.003 422.659,293.399 425.456,295.782 428.253,298.138 
 431.05,300.457 433.846,302.725 436.643,304.935 439.44,307.077 442.237,309.143 445.034,311.128 447.831,313.026 450.628,314.833 453.424,316.546 456.221,318.163 
 459.018,319.685 461.815,321.11 464.612,322.439 467.409,323.675 470.205,324.819 473.002,325.874 475.799,326.843 478.596,327.731 481.393,328.541 484.19,329.277 
 486.986,329.943 489.783,330.545 492.58,331.085 495.377,331.57 498.174,332.003 500.971,332.388 503.767,332.729 506.564,333.031 509.361,333.297 512.158,333.53 
 514.955,333.734 517.752,333.912 520.548,334.066 523.345,334.2 526.142,334.316 528.939,334.415 531.736,334.5 534.533,334.573 537.329,334.635 540.126,334.688 
 542.923,334.732 545.72,334.77 548.517,334.802 551.314,334.828 554.11,334.85 556.907,334.869 559.704,334.884 562.501,334.897 565.298,334.907 568.095,334.916 
 570.891,334.923 573.688,334.928 576.485,334.933 579.282,334.937 582.079,334.94 584.876,334.942 587.672,334.944 590.469,334.946 593.266,334.947 596.063,334.948 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.7; stroke-opacity:1; fill:none" points="
 39.4926,334.952 42.2894,334.952 45.0862,334.952 47.8831,334.952 50.6799,334.952 53.4767,334.952 56.2736,334.952 59.0704,334.952 61.8673,334.952 64.6641,334.952 
 67.4609,334.952 70.2578,334.952 73.0546,334.952 75.8514,334.952 78.6483,334.952 81.4451,334.952 84.2419,334.952 87.0388,334.952 89.8356,334.952 92.6325,334.952 
 95.4293,334.952 98.2261,334.

# Plots

In [123]:
kdenpoints = 100
trajts = 0:1/5:20
sols = [GynC.Federn.odesol(k,trajts, m=mass) for k = xs]
meas = [-8]

function bayesposterior(m, data, wprior)
    L = likelihoodmat(m.ys, data, m.measerr)
    GynC.emiteration(wprior, L)
end


function plottrajdens(ts, sols, w)

  msol = hcat(sols...)
  mkde = hcat([KernelDensity.kde(msol[t,:], boundary = (-13,13), weights=w, bandwidth = 0.5, npoints=kdenpoints).density for t in 1:size(msol, 1)]...)

  contour(ts, linspace(-13,13, kdenpoints), mkde, clims=(1e-5,0.3), fill=true, seriescolor = :heat, legend=false)
end


function plotcol(w)
    
    yscale = length(xs) / maximum(xs)
    ylims = (0,.15)
    
    pprior = plot(xs, w*yscale, legend=false, color="black", ylims=ylims)
    
    wpost = bayesposterior(m, meas, w)
    
    plot!(xs, wpost*yscale, color=:dodgerblue)
    
    
    pprioq = plottrajdens(trajts, sols, w)
    scatter!(pprioq, repmat(tmeas, length(m.datas)), m.datas, alpha = 0.2, markerstrokewidth = 0, marker=:hline, color=:black)
    
    ppostq = plottrajdens(trajts, sols, wpost)
    scatter!(ppostq, tmeas, meas, marker=:hline, color=:dodgerblue, ms=6)
    [pprior, pprioq, ppostq]
end

#plot(plotcol(ws["NPMLE"])..., size=(1000,400), layout=(1,3))

plotcol (generic function with 1 method)

In [124]:
ws = wsgrad

meas = [-8]
aplots = hcat(plotcol(wprior), plotcol(w0), plotcol(ws["NPMLE"][end]), plotcol(ws["DS-MLE"][end]), plotcol(ws["MPLE"][end]))
plot(aplots..., size=(1200,900), layout=(5,3))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
<polyline clip-path="url(#clip02)" style="stroke:#00002d; stroke-width:1.8; stroke-opacity:1; fill:none" points="
 48.3815,164.952 50.1882,164.952 51.9949,164.952 53.8016,164.952 55.6083,164.952 57.415,164.952 59.2217,164.952 61.0284,164.952 62.8351,164.952 64.6418,164.952 
 66.4485,164.952 68.2552,164.952 70.0619,164.951 71.8686,164.951 73.6753,164.95 75.482,164.949 77.2887,164.948 79.0954,164.945 80.9021,164.941 82.7088,164.934 
 84.5155,164.924 86.3222,164.909 88.1289,164.887 89.9356,164.854 91.7423,164.806 93.549,164.737 95.3557,164.639 97.1624,164.501 98.9691,164.311 100.776,164.053 
 102.582,163.704 104.389,163.241 106.196,162.634 108.003,161.848 109.809,160.847 111.616,159.586 113.423,158.023 115.229,156.109 117.036,153.802 118.843,151.061 
 120.649,147.851 122.456,144.15 124.263,139.949 126.07,135.258 127.876,130.106 129.683,124.547 131.49,118.658 133.296,112.543 135.103,106.325 136.91,100.149 
 138.716,94.175 140.523,88.5691 142.33,83.4988 144.137,79.1238 145.943,75.5877 147.75,73.0099 149.557,71.4791 151.363,71.0479 153.17,71.7299 154.977,73.4986 
 156.783,76.2889 158.59,80.0011 160.397,84.5065 162.204,89.6542 164.01,95.2798 165.817,101.213 167.624,107.287 169.43,113.343 171.237,119.241 173.044,124.858 
 174.85,130.098 176.657,134.886 178.464,139.174 180.271,142.935 182.077,146.162 183.884,148.866 185.691,151.07 187.497,152.808 189.304,154.117 191.111,155.042 
 192.917,155.624 194.724,155.906 196.531,155.926 198.338,155.721 200.144,155.32 201.951,154.75 203.758,154.035 205.564,153.194 207.371,152.241 209.178,151.192 
 210.984,150.055 212.791,148.842 214.598,147.56 216.405,146.218 218.211,144.821 220.018,143.379 221.825,141.898 223.631,140.386 225.438,138.851 227.245,137.302 
 229.051,135.748 230.858,134.198 232.665,132.663 234.472,131.152 236.278,129.676 238.085,128.246 239.892,126.872 241.698,125.565 243.505,124.334 245.312,123.19 
 247.119,122.143 248.925,121.2 250.732,120.37 252.539,119.66 254.345,119.077 256.152,118.625 257.959,118.308 259.765,118.129 261.572,118.091 263.379,118.192 
 265.186,118.433 266.992,118.811 268.799,119.323 270.606,119.964 272.412,120.728 274.219,121.61 276.026,122.6 277.832,123.692 279.639,124.876 281.446,126.143 
 283.253,127.481 285.059,128.882 286.866,130.334 288.673,131.827 290.479,133.351 292.286,134.894 294.093,136.448 295.899,138.002 297.706,139.547 299.513,141.076 
 301.32,142.579 303.126,144.051 304.933,145.484 306.74,146.873 308.546,148.213 310.353,149.5 312.16,150.731 313.966,151.903 315.773,153.014 317.58,154.063 
 319.387,155.05 321.193,155.974 323,156.836 324.807,157.638 326.613,158.38 328.42,159.064 330.227,159.693 332.033,160.269 333.84,160.794 335.647,161.271 
 337.454,161.703 339.26,162.093 341.067,162.444 342.874,162.758 344.68,163.039 346.487,163.289 348.294,163.51 350.1,163.706 351.907,163.878 353.714,164.03 
 355.521,164.162 357.327,164.277 359.134,164.377 360.941,164.464 362.747,164.539 364.554,164.604 366.361,164.659 368.167,164.706 369.974,164.746 371.781,164.781 
 373.588,164.809 375.394,164.834 377.201,164.854 379.008,164.872 380.814,164.886 382.621,164.898 384.428,164.908 386.234,164.916 388.041,164.923 389.848,164.928 
 391.655,164.933 393.461,164.937 395.268,164.94 397.075,164.942 398.881,164.944 400.688,164.946 402.495,164.947 404.301,164.948 406.108,164.949 407.915,164.949 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#1d8fff; stroke-width:1.8; stroke-opacity:1; fill:none" points="
 48.3815,164.952 50.1882,164.952 51.9949,164.952 53.8016,164.952 55.6083,164.952 57.415,164.952 59.2217,164.952 61.0284,164.952 62.8351,164.952 64.6418,164.952 
 66.4485,164.952 68.2552,164.952 70.0619,164.952 71.8686,164.952 73.6753,164.952 75.482,164.952 77.2887,164.952 79.0954,164.952 80.9021,164.952 82.7088,164.952 
 84.5155,164.952 86.3222,164.952 8

In [125]:
savefig("paper federn.pdf")

In [15]:
r(x) = GynC.Federn.maxtreatamplitude(x) > 13 ? 1 : 0

rxs = r.(xs)

R(w, data) = dot(bayesposterior(m, data, w), rxs)

stdR(w) = sqrt(1/(length(m.datas)-1) * sumabs2((R(w, data) - R(wprior, data)) for data in m.datas))

map(stdR, [w0, ws["NPMLE"][end], ws["DS-MLE"][end], ws["MPLE"][end]])

computing likelihood matrix (148x1)

 (0.002945622 seconds)


LoadError: TypeError: likelihoodmat_unmemoized: in typeassert, expected Array{Float64,2}, got Array{Float64,1}

In [84]:
eval(GynC, :(OPT_XTOLABS = 0))
eval(GynC, :(OPT_XTOLREL = 0))
eval(GynC, :(OPT_MAXEVAL = 0))
eval(GynC, :(OPT_FTOLREL = 1e-8))
eval(GynC, :(OPT_CTOLABS = 1e-6));
eval(GynC, :(OPT_DEBUG = true));